# Resource Validation

In [8]:
LOCAL="http://localhost:8088/fhir/" # Jupyter uses 8888!!
JHU="http://20.119.216.32:8000/r4/"
server_path=LOCAL

from imports import ResourceValidation
client = ResourceValidation(LOCAL, False, False)
#client = TimsClient(BASE_URL, False, True)
# client = TimsClient(BASE_URL, True, True)

## Organization  

In [9]:
client.put_organization("0_organization.json")

http://localhost:8088/fhir//Organization/?_format=json&_pretty=true


In [10]:
client.get_organization("HealthGorilla")

name:HealthGorilla
resourceType: Organization  id: 18829



## Patient

In [11]:
client.put_patient("1_simple_patient.json")

http://localhost:8088/fhir//Patient/?_format=json&_pretty=true


In [12]:
client.get_patient("Chalmers")

name:Chalmers
name:Peter
resourceType: Patient  id: 18845



## Encounter

In [13]:
client.put_encounter("2_encounter.json")

http://localhost:8088/fhir//Encounter/?_format=json&_pretty=true


In [14]:
client.get_encounter('18845')

http://localhost:8088/fhir/Encounter/18852
id: 18852
text status: generated
text div: <div xmlns="http://www.w3.org/1999/xhtml">Encounter with patient</div>
status: in-progress

